In [41]:
import pickle
from collections import defaultdict

In [42]:
di="./debug/"

In [43]:
! ls  $di

 1_src_step1_mstep0.pickle          8_src_step3_mstep1.pickle
 1_tgt_step1_mstep0.pickle          8_tgt_step3_mstep1.pickle
 2_src_step1_mstep1.pickle          9_src_step3_prune.pickle
 2_tgt_step1_mstep1.pickle          9_src_step3_pruneloss.pickle
 3_src_step1_prune.pickle          10_src_step3_pruneloss_diff.pickle
 3_src_step1_pruneloss.pickle      10_src_step4_mstep0.pickle
 4_src_step1_pruneloss_diff.pickle 10_tgt_step3_pruneloss_diff.pickle
 4_src_step2_mstep0.pickle         10_tgt_step4_mstep0.pickle
 4_tgt_step1_pruneloss_diff.pickle 11_src_step4_mstep1.pickle
 4_tgt_step2_mstep0.pickle         11_tgt_step4_mstep1.pickle
 5_src_step2_mstep1.pickle         12_src_step4_prune.pickle
 5_tgt_step2_mstep1.pickle         12_src_step4_pruneloss.pickle
 6_src_step2_prune.pickle          13_src_step4_pruneloss_diff.pickle
 6_src_step2_pruneloss.pickle      13_src_step5_mstep0.pickle
 7_src_step2_pruneloss_diff.pickle 13_tgt_step4_pruneloss_diff.pickle
 7_src_step3_mstep0.pickle        

In [44]:
all_files  =! ls $di
pruned_voc_files = !ls $di | grep pruneloss | grep -v diff
pruned_alignloss_diff = !ls $di | grep pruneloss_diff

In [45]:
print(all_files)
print(pruned_voc_files)
print(pruned_alignloss_diff)

[' 1_src_step1_mstep0.pickle', ' 1_tgt_step1_mstep0.pickle', ' 2_src_step1_mstep1.pickle', ' 2_tgt_step1_mstep1.pickle', ' 3_src_step1_prune.pickle', ' 3_src_step1_pruneloss.pickle', ' 4_src_step1_pruneloss_diff.pickle', ' 4_src_step2_mstep0.pickle', ' 4_tgt_step1_pruneloss_diff.pickle', ' 4_tgt_step2_mstep0.pickle', ' 5_src_step2_mstep1.pickle', ' 5_tgt_step2_mstep1.pickle', ' 6_src_step2_prune.pickle', ' 6_src_step2_pruneloss.pickle', ' 7_src_step2_pruneloss_diff.pickle', ' 7_src_step3_mstep0.pickle', ' 7_tgt_step2_pruneloss_diff.pickle', ' 7_tgt_step3_mstep0.pickle', ' 8_src_step3_mstep1.pickle', ' 8_tgt_step3_mstep1.pickle', ' 9_src_step3_prune.pickle', ' 9_src_step3_pruneloss.pickle', '10_src_step3_pruneloss_diff.pickle', '10_src_step4_mstep0.pickle', '10_tgt_step3_pruneloss_diff.pickle', '10_tgt_step4_mstep0.pickle', '11_src_step4_mstep1.pickle', '11_tgt_step4_mstep1.pickle', '12_src_step4_prune.pickle', '12_src_step4_pruneloss.pickle', '13_src_step4_pruneloss_diff.pickle', '13_s

In [46]:
pickles=[]

In [47]:
for file in files:
    break
    with open(di+file,"rb") as f:
        pickles.append(pickle.load(f))

In [48]:
prune_voc_pickles=[]
for file in pruned_voc_files:
    with open(di+file,"rb") as f:
        prune_voc_pickles.append(pickle.load(f))

In [ ]:
pickles[0].keys()

In [50]:
#alignlossの方が桁が大きい。-.LM lossに比べて影響が大きい
for i,pic in enumerate(prune_voc_pickles):
    remain_avg_align_loss= sum([pic["remain"][v]["Align_loss"] for v in pic["remain"]])/len(pic["remain"])
    remain_avg_LM_loss= sum([pic["remain"][v]["LM_loss"] for v in pic["remain"]])/len(pic["remain"])
    remove_avg_align_loss= sum([pic["remove"][v]["Align_loss"] for v in pic["remove"]])/len(pic["remove"])
    remove_avg_LM_loss= sum([pic["remove"][v]["LM_loss"] for v in pic["remove"]])/len(pic["remove"])
    print("step {}######".format(i))
    print("remain_avg_align_loss=>",remain_avg_align_loss)
    print("remain_avg_LM_loss=>",remain_avg_LM_loss)
    print("#")
    print("remove_avg_align_loss=>",remove_avg_align_loss)
    print("remove_avg_LM_loss=>",remove_avg_LM_loss)

step 0######
remain_avg_align_loss=> 0.4378783352063113
remain_avg_LM_loss=> 0.005495928346054029
#
remove_avg_align_loss=> -0.11713009882461176
remove_avg_LM_loss=> 0.0036760845746887376
step 1######
remain_avg_align_loss=> 0.6223537353173785
remain_avg_LM_loss=> 0.0076719140877247955
#
remove_avg_align_loss=> -0.022972800890811593
remove_avg_LM_loss=> 0.006334314678028819
step 2######
remain_avg_align_loss=> 0.7812561370063358
remain_avg_LM_loss=> 0.010733687362600632
#
remove_avg_align_loss=> 0.22965644555342798
remove_avg_LM_loss=> 0.007950188180445333
step 3######
remain_avg_align_loss=> 0.8219923854191028
remain_avg_LM_loss=> 0.012368048506000996
#
remove_avg_align_loss=> 0.17531647688078883
remove_avg_LM_loss=> 0.009315863993653705


In [51]:
#長い単語ほど、alignlossの値が大きい。->長い単語が残りやすい(freqが多いほどalignlossが小さくなるとかになってる?)
for i,pic in enumerate(prune_voc_pickles):
    remain_align_len=defaultdict(list)
    remain_LM_len = defaultdict(list)
    remove_align_len=defaultdict(list)
    remove_LM_len=defaultdict(list)
    for v in pic["remain"]:
        remain_align_len[len(v)].append(pic["remain"][v]["Align_loss"])
        remain_LM_len[len(v)].append(pic["remain"][v]["LM_loss"])
    
    for v in  pic["remove"]:
        remove_align_len[len(v)].append(pic["remove"][v]["Align_loss"])
        remove_LM_len[len(v)].append(pic["remove"][v]["LM_loss"])
    remain_align_res = list(sorted([(v,"len{} av{}".format(v,sum(remain_align_len[v])/len(remain_align_len[v]))) for v in remain_align_len.keys()]))
    remain_LM_res = list(sorted([(v,"len{} av{}".format(v,sum(remain_LM_len[v])/len(remain_LM_len[v]))) for v in remain_LM_len.keys()]))
    print("align:"+"\n".join([v[1] for v  in remain_align_res]))
    print("LM:"+"\n".join([v[1] for v  in remain_LM_res]))

align:len2 av0.4266061309193182
len3 av0.42945554782295003
len4 av0.44177875913933795
len5 av0.463372294718772
len6 av0.4437014773647111
len7 av0.46691097150637506
len8 av0.4523409723724124
len9 av0.4253532477497956
len10 av0.5357500318518459
len11 av0.4683683836011656
len12 av0.44710136062750583
len13 av0.8541713540936249
len14 av0.8055494293814615
len15 av0.4176313406497213
len16 av0.819106780400218
LM:len2 av0.006189741132526398
len3 av0.005124237874729189
len4 av0.005071922935776316
len5 av0.0047078087263038165
len6 av0.004611327431059754
len7 av0.005597485516424461
len8 av0.005587804268068721
len9 av0.006486834829820225
len10 av0.008767246123724521
len11 av0.008922687776573679
len12 av0.014441641712333166
len13 av0.020467682521656153
len14 av0.014330535156651433
len15 av0.011424449291950235
len16 av0.02259368028428701
align:len2 av0.5447284607118958
len3 av0.627842870178361
len4 av0.6654625045751601
len5 av0.6862873699819902
len6 av0.7037396566316946
len7 av0.7163737348037086
len8

In [52]:
#prune_voc_pickles[3]

In [53]:
pruned_alignloss_diff_pickels=[]
for file in pruned_alignloss_diff:
    with open(di+file,"rb") as f:
        pruned_alignloss_diff_pickels.append(pickle.load(f))

In [54]:
#scoreわるくなってるのでは。(長さの正説があるので、長さでわる)
for i,pic in enumerate(pruned_alignloss_diff_pickels):
    print(pic)

{'algin_before': -2.8967265260176704, 'align_after': -2.8825625971597733, 'gain': 0.01416392885789719}
{'algin_before': -2.6656178218899838, 'align_after': -2.8825625971597733, 'gain': -0.2169447752697895}
{'algin_before': -2.8752452705190383, 'align_after': -2.8867395176224147, 'gain': -0.01149424710337632}
{'algin_before': -2.7311852444384837, 'align_after': -2.8867395176224147, 'gain': -0.155554273183931}
{'algin_before': -2.884226044704115, 'align_after': -2.9411083332937196, 'gain': -0.056882288589604535}
{'algin_before': -2.778552143588447, 'align_after': -2.9411083332937196, 'gain': -0.16255618970527275}
{'algin_before': -2.943428515822479, 'align_after': -2.9517435558777967, 'gain': -0.008315040055317802}
{'algin_before': -2.8127533547053596, 'align_after': -2.9517435558777967, 'gain': -0.1389902011724371}
